In [1]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

# Virtual Memory

What if we run the following program on a machine with 32GB memory?

In [3]:
render_code("memory_allocation.c")

// memory_allocation.c:1-37 (37 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>

//#define dim 32768
//#define dim 49152
int main(int argc, char *argv[])
{
    int i,j;
    double *a;
    double sum=0, average;
    int dim=32768;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s dimension\n",argv[0]);
        exit(1);
    }
    dim = atoi(argv[1]);
    a = (double *)malloc(sizeof(double)*dim*dim);
//    for(i = 0 ; i < dim; i++)
//        a[i] = (double *)malloc(sizeof(double)*dim);
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            a[i*dim+j] = rand();
    for(i = 0 ; i < dim; i++)
        for(j = 0 ; j < dim; j++)
            sum+=a[i*dim+j];
    average = sum/(dim*dim);
    fprintf(stderr,"average: %lf\n",average);
    free(a);
    return 0;
}

In [9]:
! cs203 job memory "free -m ; make ./memory_allocation; ./memory_allocation 70000"

srun -N1 -p cs203m free -m 
              total        used        free      shared  buff/cache   available
Mem:          32052         936       30231           2         884       30663
Swap:         62595         976       61619
srun -N1 -p cs203m  make ./memory_allocation
srun: job 20576 queued and waiting for resources
srun: job 20576 has been allocated resources
make: 'memory_allocation' is up to date.
srun -N1 -p cs203m  ./memory_allocation 70000
srun: job 20577 queued and waiting for resources
srun: job 20577 has been allocated resources
average: 8696013447.584116


What's the output of the following code if we run 4 instances at the same time?

In [10]:
render_code("virtualization.c")

// virtualization.c:1-28 (28 lines)
#define _GNU_SOURCE  
#include <unistd.h>
#include <stdio.h>
#include <stdlib.h>
#include <assert.h>
#include <sched.h>
#include <sys/syscall.h>
#include <time.h>

double a = 0;

int main(int argc, char *argv[])
{
    int i, number_of_total_processes=4;
    if(argc < 2)
    {
        fprintf(stderr, "Usage: %s number_of_processes\n",argv[0]);
        exit(1);
    }
    number_of_total_processes = atoi(argv[1]);
    for(i = 0; i< number_of_total_processes-1 && fork(); i++);
    srand((int)time(NULL)+(int)getpid());
    a = rand();
    fprintf(stderr, "\nProcess %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    sleep(10);
    fprintf(stderr, "\nProcess %d: Value of a is %lf and address of a is %p\n",(int)getpid(), a, &a);
    return 0;
}

In [11]:
! make ; ./virtualization 4 

make: Nothing to be done for 'all'.

Process 2563390: Value of a is 1102604738.000000 and address of a is 0x601090

Process 2563389: Value of a is 1401783322.000000 and address of a is 0x601090

Process 2563391: Value of a is 1869222550.000000 and address of a is 0x601090

Process 2563392: Value of a is 1566646015.000000 and address of a is 0x601090

Process 2563391: Value of a is 1869222550.000000 and address of a is 0x601090

Process 2563390: Value of a is 1102604738.000000 and address of a is 0x601090

Process 2563392: Value of a is 1566646015.000000 and address of a is 0x601090

Process 2563389: Value of a is 1401783322.000000 and address of a is 0x601090
